In [2]:
using Pkg

Pkg.activate("FluxModuleDev")

  Activating project at `~/Desktop/CS/JuliaTransformers/FluxTransformerModules/FluxModuleDev`


In [138]:
begin 
    Pkg.add("Flux")
    Pkg.add("LinearAlgebra")
    Pkg.add("CUDA")
    Pkg.add("Pipe")
    Pkg.add("BenchmarkTools")
end

   Resolving package versions...


  No Changes to `~/Desktop/CS/JuliaTransformers/FluxTransformerModules/FluxModuleDev/Project.toml`
  No Changes to `~/Desktop/CS/JuliaTransformers/FluxTransformerModules/FluxModuleDev/Manifest.toml`


   Resolving package versions...


  No Changes to `~/Desktop/CS/JuliaTransformers/FluxTransformerModules/FluxModuleDev/Project.toml`
  No Changes to `~/Desktop/CS/JuliaTransformers/FluxTransformerModules/FluxModuleDev/Manifest.toml`


   Resolving package versions...


...

Precompiling project...


  ✓ Parsers


  ✓ JSON


  ✓ BenchmarkTools


  3 dependencies successfully precompiled in 4 seconds (95 already precompiled)


In [25]:
using Flux
using BenchmarkTools
using LinearAlgebra
using CUDA
using Pipe


ArgumentError: ArgumentError: Package NNlib not found in current path:
- Run `import Pkg; Pkg.add("NNlib")` to install the NNlib package.


In [4]:
const Q = [1 1 2 2 3 3 ; 1 1 3 3 4 6 ; 1 3 4 5 6 7]
const K = [1 2 3 2 6 5 ; 0 4 4 5 6 5 ; 4 6 5 8 6 4]

3×6 Matrix{Int64}:
 1  2  3  2  6  5
 0  4  4  5  6  5
 4  6  5  8  6  4

In [5]:
function sdpa(Q::AbstractArray, K::AbstractArray, V::AbstractArray)
    size(Q) == size(K) ? nothing : AssertionError("Dimension of query (Q) and keys (K) must match!") |> throw
    dₖ =  size(K)[1]
    Kᵀ = transpose(K)
    return (softmax(Q * Kᵀ) / √dₖ) * V
end

sdpa (generic function with 1 method)

In [51]:
function windowedattn(Q::AbstractArray, K::AbstractArray, blocksize::Int, window::Int)

    Q_dims = size(Q)
    K_dims = size(K)
    Q_dims[2] % blocksize == 0 ? (num_blocks = (Q_dims[2] / blocksize) |> Integer) : AssertionError("Block size must divide query length!") |> throw
    window % 2 == 0 ? AssertionError("Window size must be odd!") |> throw : nothing
    Q_dims == K_dims ? nothing : AssertionError("Dimension of query (Q) and keys (K) must match!") |> throw
    dₖ =  K_dims[1]

    block_Q = @pipe reshape(Q, (dₖ, blocksize, num_blocks)) |> permutedims(_, (2, 1, 3))
    block_K = @pipe reshape(K, (dₖ, blocksize, num_blocks)) |> permutedims(_, (1, 2, 3))

    shift = (window-1)/2 
    rolled_K = [circshift(block_K, n) for n ∈ range(-shift,shift)]
    
    return [batched_mul(block_Q, Kᵣ) for Kᵣ ∈ rolled_K]

end

windowedattn (generic function with 1 method)

In [52]:
@btime windowedattn($Q,$K,2,3) 

  4.907 μs (95 allocations: 189.42 KiB)


3-element Vector{Array{Int64, 3}}:
 [5 12; 7 16;;; 35 42; 38 44;;; 78 61; 96 74]
 [5 12; 13 24;;; 38 51; 43 59;;; 78 59; 96 73]
 [5 12; 5 20;;; 35 42; 39 47;;; 78 62; 96 77]

In [49]:
attn[1]

2×2×3 Array{Int64, 3}:
[:, :, 1] =
 5  12
 7  16

[:, :, 2] =
 35  42
 38  44

[:, :, 3] =
 78  61
 96  74

In [62]:
rolls(5)

5-element Vector{Float64}:
 -2.0
 -1.0
  0.0
  1.0
  2.0

In [58]:
x = circshift(1 2;3 4 ;3 6],2)

3×2 Matrix{Int64}:
 3  4
 3  6
 1  2

4-element Vector{Int64}:
 3
 4
 1
 2

In [37]:
K = [1 2 ; 0 4 ]
V = [1 3 4 ; 1 3 2]
V = reshape(V,(3,:))

3×2 Matrix{Int64}:
 1  3
 1  4
 3  2

In [32]:
sdpa(Q,K,V)

2×3 Matrix{Float64}:
 0.707107  2.12132  2.12132
 0.707107  2.12132  2.12132